# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key 

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = os.path.join('../PythonAPI-Challenge/CSVs/cities.csv')

city_data = pd.read_csv(csv_path)
city_data.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = city_data[["Lat", "Lng"]]
humidity = city_data["Humidity"].astype(float)

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Max Temp of 80, higher than 70
# Wind less than 10 mph
# 0 cloudiness

hotel_df = city_data.loc[(city_data['Max Temp'] <= 80) & (city_data['Max Temp'] >= 70) &
                             (city_data['Wind Speed'] <= 10) & (city_data['Cloudiness'] == 0)]
len(hotel_df)

8

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df["Hotel Name"] = ""
hotel_df.head()

/var/folders/5b/vwmptz0s5wx33k249kc8dmjw0000gn/T/ipykernel_13634/1149337638.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44,
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,


In [7]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():

    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    # make request and print url
    hotel_info = requests.get(base_url, params=params)
    
    # convert to json
    hotel_info = hotel_info.json()
    print(json.dumps(hotel_info, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_info["results"][0]["name"]
        

    except (KeyError, IndexError, TypeError):
        print("Missing field/result... skipping.")

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -18.0787379,
                    "lng": 13.8845578
                },
                "viewport": {
                    "northeast": {
                        "lat": -18.07804977010728,
                        "lng": 13.88543002989272
                    },
                    "southwest": {
                        "lat": -18.08074942989272,
                        "lng": 13.88273037010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Kaoko Mopane Lodge & Campsite | Opuwo",
            "opening_hours": {
                "open_no

/Users/ap20/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


{
    "html_attributions": [],
    "next_page_token": "Aap_uED0mVUptWceGO8HpRBQIGVGcqdRd2SCYqMCV8dwk08vr4dcVqDaYfUuhbvip3CwOOEqFOyscluNuYPgeC4-nRpHi14dgtz_GPqB8t-ZocbQG_LjOkEO82fhpuKGxQa-FTbNBBvOVKiCZpQPX9mC9LU86iSwl5p7NpBYgQEad8kQIlbXKa80oBfxAQh9Pnpf_jzWon-1cvTZoZWDBGiajeznp9ri4I1GIXxio4AGE8HGvv8dzII-eHScr_C1CaQB_xjJ0i5DD9dgPP-1fZB4XcoIrjkIfcpvwcv-xWK_vdjnugLETAonm-e4MikSpX2R5Lw03fTZgwmxDrDbV0NJvSM6Cv5WYhiCzsENqCnLnSK2qnMlK3HdTpY3EDvI01ZHOe6JJfJnSteWKPunjs_X8uyCwq4OZfbLCIrwX_yOh4ZzXLjErybor73Q0Cwnj2w",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.6289865,
                    "lng": -46.0285662
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.62777447010728,
                        "lng": -46.02738632010728
                    },
                    "southwest": {
                        "lat": -20.63047

In [8]:
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44,Kaoko Mopane Lodge & Campsite | Opuwo
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Escarpas Resort
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Hotel Castelo
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,Palissandre Cote Ouest resort & SPA
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Dandeli Aura Jungle Stay
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Hôtel Saja
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,Mount Mogok Hotel
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,مهمان پذیر نوید


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
info_hotels = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations_hotel = hotel_df[["Lat", "Lng"]]

In [10]:
# Plot Marker Layer

gmaps.configure(api_key=g_key)

marker_layer = gmaps.marker_layer(locations_hotel, info_box_content=info_hotels)

fig = gmaps.figure(map_type='HYBRID')
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Plot hotel markers on top of heat map
fig = gmaps.figure(map_type='HYBRID')

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))